# Collection of implemented Greedy Algorithms for the lecture Algorithm techniques SS25 HTWG Konstanz

# A* Algorithm

The A* algorithm uses a heuristik to determine the best way to move through a graph from a starting point to an endpoint.

In [ ]:
heuristic = {
    "A": 6, 
    "B": 4,
    "C": 6,
    "D": 5,
    "E": 2,
    "F": 0
}


graph = {
    "A": [("B", 2), ("C", 4)],
    "B": [("D", 3), ("E", 2)],
    "C": [("D", 1)],
    "D": [("F", 5)],
    "E": [("F", 2)],
    "F": []
}

def pop_best(list):
   
  best = sorted(list, key=lambda x: x[1])[0]
  list.remove(best)

  return best
      

def a_star(priority_list=[('A', 0)], visited=[], curr_cost=0,start_node='A', end_node='F', optimal=[]):


    while (priority_list):

      best = pop_best(priority_list)
      visited.append(best[0])
      optimal.append(best)

      if best[0] == end_node: return optimal

      current = 0 if best[0] == start_node else best[1] - heuristic[best[0]]
      for child in graph[best[0]]:
        if child[0] in visited:
           continue

        way_cost = current + child[1]
        estimated = way_cost + heuristic[child[0]]
        priority_list.append((child[0], estimated))

best_path = a_star()
print(best_path)


[('A', 0), ('B', 6), ('E', 6), ('F', 6)]


# Greedy optimal activity selection algorithm

In [55]:
activities = [
    ("A1", 0, 6),
    ("A2", 1, 4),
    ("A3", 3, 5),
    ("A4", 5, 7),
    ("A5", 8, 9),
    ("A6", 5, 9),
    ("A7", 2, 13),
    ("A8", 6, 10),
    ("A9", 8, 11),
    ("A10", 12, 14),
    ("A11", 13, 16),
    ("A12", 0, 3),
    ("A13", 4, 5),
    ("A14", 9, 10),
    ("A15", 11, 12),
]



def optimal_plan(activities=activities):
  activities = sorted(activities, key=lambda x: x[2])
  best_activities = []
  curr_time = 0
  while list(filter(lambda x: x[1] >= curr_time, activities)):
    filtered = list(filter(lambda x: x[1] >= curr_time, activities))
    best = filtered.pop(0)
    best_activities.append(best)
    curr_time = best[2]
    
  return best_activities

res = optimal_plan()
print(res)

[('A12', 0, 3), ('A3', 3, 5), ('A4', 5, 7), ('A5', 8, 9), ('A14', 9, 10), ('A15', 11, 12), ('A10', 12, 14)]


# Approx Bin Packing

Ziel ist es eine Menge an Items mit unterschiedlicher groesse in moeglichst wenige pakete zu packen die eine fixe groese haben.

In [66]:
packets = [2, 5, 4, 7, 1, 3, 8]
bin_capacity = 10

def fits(packet, bins):
  for bin in bins:
    if sum(bin) + packet <= bin_capacity:
      return bin
    
  return None

def bin_packing(packets=packets):
  
  curr_bins = []
  packets = sorted(packets, reverse=True) # to sort the array descending the approximation get closer to optimal :)
  
  while(packets):
    packet = packets.pop(0)
    
    if not curr_bins:
      curr_bins.append([packet])
    else:
      bin = fits(packet, curr_bins)
      bin.append(packet) if bin else curr_bins.append([packet])
  return curr_bins


res = bin_packing()
print(res) 

[[8, 2], [7, 3], [5, 4, 1]]


# Kruskal's Greedy Based MST algorithm

Ziel ist es einen Minimal spannenden Baum ueber alle Knoten eines Graphen zu bilden.


In [82]:
edges = [
    ("A", "B", 1),
    ("A", "C", 3),
    ("A", "D", 4),
    ("B", "C", 2),
    ("C", "D", 5),
]

edges2 = [
    ("A", "B", 4),
    ("A", "H", 8),
    ("B", "C", 8),
    ("B", "H", 11),
    ("C", "D", 7),
    ("C", "F", 4),
    ("C", "I", 2),
    ("D", "E", 9),
    ("D", "F", 14),
    ("E", "F", 10),
    ("F", "G", 2),
    ("G", "H", 1),
    ("G", "I", 6),
    ("H", "I", 7),
    ("B", "E", 6),
]

sub_trees = [[node] for node in sorted({n for edge in edges2 for n in edge[:2]})]
min_tree = []

def merge_trees(edge):
  t1 = next(filter(lambda x: edge[0] in x,sub_trees))
  t2 = next(filter(lambda x: edge[1] in x,sub_trees))

  sub_trees.remove(t1)
  sub_trees.remove(t2)
  sub_trees.append(t1 + t2)

def is_cycle(edge, sub_trees):

  for tree in sub_trees:
    if edge[0] in tree and edge[1] in tree:
      return True
  return False


def mst(edges, min_tree, sub_trees):
  edges = sorted(edges, key=lambda x: x[2])
  while edges:
    min_edge = edges.pop(0)
    if is_cycle(min_edge, sub_trees):
      continue
    min_tree.append(min_edge)
    merge_trees(min_edge)

  return min_tree

res = mst(edges=edges2, min_tree=[], sub_trees=sub_trees)
print(res)

cost = sum(w for _, _, w in res)
print(cost)

[('G', 'H', 1), ('C', 'I', 2), ('F', 'G', 2), ('A', 'B', 4), ('C', 'F', 4), ('B', 'E', 6), ('C', 'D', 7), ('A', 'H', 8)]
34
